# Librerie

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

import re
import hexdump
import sklearn as sk
import numpy as np

import matplotlib.pyplot as plt

2023-05-25 15:37:47.776986: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Caricamento e descrizione Dataset

Caricamento del dataset [room_occupancy](https://www.kaggle.com/code/ashrafsharifi/exploratorydataanalysisproject) per il rilevamento di persone in una stanza in funzione di determinati valori (temperatura, umidità, luce, CO2). 

In [5]:
df = pd.read_csv('dataset/room_occupancy.csv')

Descrizione del dataset: 
20560 samples, ciascuno contenente 6 feature (data, 

*   20560 samples;
*   6 feature per sample (data, tempertura, umidità, luce, CO2, ratio umidità);
*   ogni feature contiene il relativo valore di occupazione (1 oppure 0).

Si è visto che non sono presenti valori nulli o nan.

Si è visto che le classi non sono perfettamente bilanciate (15810 sample con valore di Occupancy a 0 e 4750 a 1). Sarà quindi necessario utilizzare, oltre che all'accuratezza, anche la precision e recall come metriche.

In [6]:
# info
print(df.info())
print("\n")

# numero si sample per valore di occupancy
print("Numero di sample per valore di Occupancy (1 oppure 0):")
print(df["Occupancy"].value_counts())

# controllo null / nan
print("\nValori nulli per colonna: \n", df.isnull().sum())
print("\nValori nan per colonna: \n", df.isnull().sum())

# visualizzo i primi elementi del dataset
print("\nPrimi elementi del dataset: \n")
print(df.head())

# visualizzo alcuni elemnti del dataset quando non è occpuata la stanza
print("\nPrimi elementi del dataset con Occupancy a zero: \n")
print(df.loc[df["Occupancy"] == 0].tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20560 entries, 0 to 20559
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           20560 non-null  object 
 1   Temperature    20560 non-null  float64
 2   Humidity       20560 non-null  float64
 3   Light          20560 non-null  float64
 4   CO2            20560 non-null  float64
 5   HumidityRatio  20560 non-null  float64
 6   Occupancy      20560 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.1+ MB
None


Numero di sample per valore di Occupancy (1 oppure 0):
Occupancy
0    15810
1     4750
Name: count, dtype: int64

Valori nulli per colonna: 
 date             0
Temperature      0
Humidity         0
Light            0
CO2              0
HumidityRatio    0
Occupancy        0
dtype: int64

Valori nan per colonna: 
 date             0
Temperature      0
Humidity         0
Light            0
CO2              0
HumidityRatio    

Split train-test: considero il 75% di dati per il train e il restante 25% per il test. Avviene in automatico uno shuffle dei dati.

In [7]:
X = df[["Temperature", "Humidity", "Light", "CO2", "HumidityRatio"]]
Y = df[["Occupancy"]]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

print("Numero di sample nel train set per valore di Occupancy (1 oppure 0):")
print(Y_train["Occupancy"].value_counts())

print("\nNumero di sample nel test set per valore di Occupancy (1 oppure 0):")
print(Y_test["Occupancy"].value_counts())

Numero di sample nel train set per valore di Occupancy (1 oppure 0):
Occupancy
0    11822
1     3598
Name: count, dtype: int64

Numero di sample nel test set per valore di Occupancy (1 oppure 0):
Occupancy
0    3988
1    1152
Name: count, dtype: int64


# Creazione NN

Di seguito avviene la [creazione della rete neurale](https://www.analyticsvidhya.com/blog/2021/11/neural-network-for-classification-with-tensorflow/) che si occuperà di capire se nella stanza sono presenti delle persone oppure no in funzione dei valori di temperatura, umidità, luce e CO2 presenti.  

Si tratta di un problema di classificazione binaria e lo si vuole modellare tramite una semplice rete neurale ([sigmoide come funzione di attivazione dell'ultimo layer e crossentropy binaria come loss](https://www.dlology.com/blog/how-to-choose-last-layer-activation-and-loss-function/)).

Sono create 3 reti in ordine di complessità.


In [17]:
tf.random.set_seed(42)

models = { 
  "small": tf.keras.Sequential(
    [ 
      tf.keras.layers.Dense(20, activation='relu', input_shape=(5,)),
      tf.keras.layers.Dense(1, activation='sigmoid'),
    ]
  ),
  "medium": tf.keras.Sequential(
    [ 
      tf.keras.layers.Dense(20, activation='relu', input_shape=(5,)),
      tf.keras.layers.Dense(50, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid'),
    ]
  ),
  "large": tf.keras.Sequential(
    [ 
      tf.keras.layers.Dense(20, activation='relu', input_shape=(5,)),
      tf.keras.layers.Dense(50, activation='relu'),
      tf.keras.layers.Dense(80, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid'),
    ]
  ),
}

Compilazione e fit del modello.

Viene utilizzata la [binary crossentropy](https://vitalflux.com/keras-categorical-cross-entropy-loss-function/#:~:text=binary_crossentropy%3A%20Used%20as%20a%20loss,two%20or%20more%20output%20labels.) come loss function in quanto si sta parlando di classificazione binaria.  

E' inoltre settato l'ottimizzatore [Adam](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/) con un learning rate di 0.01.

In [18]:
models_history = {}
for name, model in models.items():
   model.compile( loss= tf.keras.losses.binary_crossentropy,
                  optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
                  metrics = [
                     tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                     tf.keras.metrics.Precision(name='precision'),
                     tf.keras.metrics.Recall(name='recall')
                  ])
   models_history[name] = model.fit(X_train, Y_train, epochs=25, validation_split=0.2, verbose=1, batch_size=32)

Epoch 1/25
386/386 [==============================] - 4s 6ms/step - loss: 0.9479 - accuracy: 0.9536 - precision: 0.8828 - recall: 0.9218 - val_loss: 0.3711 - val_accuracy: 0.8985 - val_precision: 0.9465 - val_recall: 0.6158
Epoch 2/25
386/386 [==============================] - 2s 5ms/step - loss: 0.6078 - accuracy: 0.9611 - precision: 0.9078 - recall: 0.9256 - val_loss: 0.6717 - val_accuracy: 0.9711 - val_precision: 0.8945 - val_recall: 0.9987
Epoch 3/25
386/386 [==============================] - 2s 5ms/step - loss: 0.4964 - accuracy: 0.9685 - precision: 0.9257 - recall: 0.9393 - val_loss: 0.3376 - val_accuracy: 0.9841 - val_precision: 0.9429 - val_recall: 0.9946
Epoch 4/25
386/386 [==============================] - 1s 4ms/step - loss: 0.3213 - accuracy: 0.9757 - precision: 0.9455 - recall: 0.9495 - val_loss: 0.2593 - val_accuracy: 0.9750 - val_precision: 0.9552 - val_recall: 0.9411
Epoch 5/25
386/386 [==============================] - 2s 6ms/step - loss: 0.3912 - accuracy: 0.9739 - pr

# Conversione e valutazione modelli

La conversione è fatta seguendo le [indicazioni ufficiali](https://www.tensorflow.org/lite/performance/post_training_quantization). Per il modello 'large' sono state fatte 2 conversioni una senza quantizzazione e l'altra con quantizzazione di tipo full-integer (anche dei tensori di input/output). 

Successivamente, il modello convertito è stato scritto come header C che poi sarà importato direttamente dal main (in PlatformIO o Arduino).  

All'interno di questo header inserisco anche l'input (che chiamo 'x') che verrà usato come test sulla scheda per simulare la predizione di un sample reale. L'input in questo caso è rappresentatao dal sample di test con id 'id_sample_to_test'. Il risultato della predizione che si ottiene sulle schede deve essere uguale a quello che si ottiene in questo notebook. 

Nell'header C inserisco anche altre due variabili interne 'input_size' e 'output_size' che indicano appunto le rispettive dimensioni di input e output.

Una volta convertiti i modelli, avviene la relativa valutazione delle performance (per i modelli Tensorflow Lite è necessario il supporto del relativo interprete). Essendo il dataset sbilanciato, si è voluto riportare, oltre ai valori di accuratezza, anche precision e recall.

In [36]:
"""
Converto il modello Tensorflow in un modello Tensoflow Lite.
E' possibile utilizzare la full-integer-quantization per ridurre la dimensione
del modello e diminuire il tempo di inferenza.

Per poterla usare, è necessario definire un dataset rappresentativo per poter definire
il range: in questo caso ho scelto 300 campioni a caso nel train.

Link utile per creazione dataset rappresentativo : https://github.com/sayakpaul/Adventures-in-TensorFlow-Lite/blob/master/Custom_Image_Classification_EdgeTPU.ipynb

Params: 
  - model: modello Tensorflow da convertire
  - full_int_quantization: bool per decidere se usare quantizzazione o no
  - x_train: dataset di train usato per la creazione del dataset rappresentativo nel caso si voglia
             fare quantizzazione

Return: modello Tensorflow Lite
"""
def convert_to_tflite(model, full_int_quantization=False, x_train=None):
  converter = tf.lite.TFLiteConverter.from_keras_model(model)

  if (full_int_quantization):

    dim = 300

    x_train_tflite = x_train.astype('float32')
    representative_sample = np.array(x_train_tflite.sample(n=dim).values.tolist(), dtype='float32')
    rep_ds = tf.data.Dataset.from_tensor_slices((representative_sample))   

    def representative_dataset():
      for sample in rep_ds.take(dim):
        yield [sample]

    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
  else:
    converter.optimizations = []

  return converter.convert()

"""
Funzione per il salvataggio di un modello Tensorflow Lite come header C che verrà
importato nel sorgente C del microcontrollore.

E' possibile inserire anche la variabile da utilizzare come input per poter testare
la correttezza della previsione del modello.
E' possibile inserire anche la dimensione degli input/output.

Params:
  - file_name: nome del file in cui salvare l'header C
  - tflite_model: modello Tensorflow Lite
  - variable_name: nome della variabile che contiene i byte del modello
  - variables_to_append: stringhe aggiuntive inserite in coda che contengono ad
      esempio il valore dell'input da testare 
"""
def save_c_header(file_name, tflite_model, variable_name="model_data", variables_to_append=""):

  bytes = hexdump.dump(tflite_model).split(' ')
  c_array = ', '.join(['0x%02x' % int(byte, 16) for byte in bytes])
  c = 'const unsigned char %s[] DATA_ALIGN_ATTRIBUTE = {%s};' % (variable_name, c_array)
  c += '\nconst int %s_len = %d;' % (variable_name, len(bytes))
  preamble = '''
    // if having troubles with min/max, uncomment the following
    // #undef min    
    // #undef max
    #ifdef __has_attribute
    #define HAVE_ATTRIBUTE(x) __has_attribute(x)
    #else
    #define HAVE_ATTRIBUTE(x) 0
    #endif
    #if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
    #define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
    #else
    #define DATA_ALIGN_ATTRIBUTE
    #endif
    '''
  open(file_name+".h", "w").write(preamble + c + variables_to_append)

"""
Avviene la valutazione delle performance di un genercico modello Tensorflow Lite.

E' necessario eseguire il modello Tensorflow Lite tramite il relativo interprete
e poi valutare il risultato delle previsioni.

E' richiesto che i float siano a 32 bit.

Params:
  - x_test: x di test su cui valutare il modello
  - y_test: y true di test per la valutazione del modello
  - tflite_model: modello Tensorflow Lite da valutare
  - io_quantized: True se tensori di input o output hanno bisogno di essere quantizzati/dequantizzati
  - id_sample_to_test: id del sample nel train set da valutare

Return: dizionario {"Prediction value of test sample with ID ": val, "Accuracy": val: "Precision": val, "Recall": val}
"""
def tflite_model_evaluate(x_test, y_test, tflite_model, io_quantized=False, id_sample_to_test=0):

  x_test_tflite = x_test.astype('float32')

  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  # quantizzo input per full-integer se serve
  if (io_quantized):
    input_scale, input_zero_point = input_details[0]["quantization"]
    x_test_tflite = x_test_tflite / input_scale + input_zero_point
    x_test_tflite = x_test_tflite.astype(input_details[0]["dtype"])

  y_pred = []

  for i in range(len(x_test_tflite.index)):
      
    interpreter.set_tensor(input_details[0]["index"], pd.DataFrame(x_test_tflite.iloc[i]).transpose())
    interpreter.invoke()
    #y_pred.append(0 if (interpreter.get_tensor(output_details[0]["index"])[0][0] < 0.5) else 1)
    y_pred.append(interpreter.get_tensor(output_details[0]["index"])[0][0])
    interpreter.reset_all_variables()

  # de-quantizzo output per full-integer se serve 
  if (io_quantized):
    y_pred = np.array(y_pred, dtype='float32')
    output_scale, output_zero_point = output_details[0]["quantization"]
    y_pred = (y_pred - output_zero_point) * output_scale

  y_pred_binary = [0 if (y < 0.5) else 1 for y in y_pred]

  acc = sk.metrics.accuracy_score(y_test, y_pred_binary)
  prec = sk.metrics.precision_score(y_test, y_pred_binary)
  rec = sk.metrics.recall_score(y_test, y_pred_binary)
  
  return {"Prediction value of test sample with ID "+str(id_sample_to_test): y_pred[id_sample_to_test], 
          "Accuracy": acc, "Precision": prec, "Recall": rec} 

"""
Avviene la valutazione del modello Tensorflow.
A differenza dei modelli Tensorflow Lite, in questo caso non serve l'interprete
ed eseguo quindi una predizione normale.

Params:
  - x_test: x di test su cui valutare il modello
  - y_test: y true di test per la valutazione del modello
  - tflite_model: modello Tensorflow Lite da valutare
  - id_sample_to_test: id del sample nel train set da valutare

Return: dizionario {"Prediction value of test sample with ID ": val, "Accuracy": val: "Precision": val, "Recall": val}
"""
def tfmodel_evaluate(x_test, y_test, tfmodel, id_sample_to_test):

  loss, acc, prec, rec = tfmodel.evaluate(X_test, Y_test)
  
  return {"Prediction value of test sample with ID "+str(id_sample_to_test): tfmodel.predict(pd.DataFrame(X_test.iloc[id_sample_to_test]).transpose() )[0][0],
          "Accuracy": acc, "Precision": prec, "Recall": rec}

"""
Funzione per printare il risultato della valutazione
"""
def print_evaluation(title, eval_dict):

  print(title)
  for k,v in eval_dict.items():
    print(k+": "+str(v))


In [38]:
# definisco la variabile che uso come input
id_sample_to_test = 7
input_length = 5
variable = "\nfloat x["+str(input_length)+"] = {"
for val in X_test.iloc[id_sample_to_test]:
  variable = variable + str(val)
  if ( not(id == input_length-1) ):        # mette la virgola a tutti tranne che l'ultimo
    variable = variable + ", "
variable += "};\n"
variable = variable + "const int input_size = "+str(input_length)+";\n"
variable = variable + "const int output_size = 1;"

for name, model in models.items():
  file_name = "c_headers/RoomOccupancyModel_"+name
  tflite_model = convert_to_tflite(model, full_int_quantization=False)
  save_c_header(file_name, tflite_model, variables_to_append=variable)
  print_evaluation("TFLite model "+name+" : ", tflite_model_evaluate(X_test, Y_test, 
                                                          tflite_model, 
                                                          io_quantized=False, 
                                                          id_sample_to_test=id_sample_to_test))

  print("\n")
  print_evaluation("TF model "+name+" : ", tfmodel_evaluate(X_test, Y_test, model, 
                                                       id_sample_to_test=id_sample_to_test))

  if (name.endswith("large")):
    file_name = "c_headers/RoomOccupancyModel_"+name+"_quant"  
    tflite_model_quant = convert_to_tflite(model, full_int_quantization=True, x_train=X_train)
    save_c_header(file_name, tflite_model_quant, variables_to_append=variable)  
    print_evaluation("TF model "+name+"_quant : ", tflite_model_evaluate(X_test, Y_test, 
                                                                         tflite_model_quant, 
                                                                         id_sample_to_test=id_sample_to_test, 
                                                                         io_quantized=True,))                                                   

INFO:tensorflow:Assets written to: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpipkasc1c/assets


INFO:tensorflow:Assets written to: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpipkasc1c/assets
2023-05-31 18:21:12.551770: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-31 18:21:12.551824: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-31 18:21:12.552650: I tensorflow/cc/saved_model/reader.cc:52] Reading SavedModel from: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpipkasc1c
2023-05-31 18:21:12.555084: I tensorflow/cc/saved_model/reader.cc:106] Reading meta graph with tags { serve }
2023-05-31 18:21:12.555127: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpipkasc1c
2023-05-31 18:21:12.603327: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-05-31 18:21:12.744425: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

TFLite model small : 
Prediction value of test sample with ID 7: 0.9905367
Accuracy: 0.9834630350194552
Precision: 0.9326845093268451
Recall: 0.9982638888888888


1/1 [==============================] - 0s 28ms/step
TF model small : 
Prediction value of test sample with ID 7: 0.9905367
Accuracy: 0.9834630489349365
Precision: 0.9326844811439514
Recall: 0.9982638955116272
INFO:tensorflow:Assets written to: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpicqrm61q/assets


INFO:tensorflow:Assets written to: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpicqrm61q/assets
2023-05-31 18:21:19.692231: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-31 18:21:19.692265: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-31 18:21:19.692556: I tensorflow/cc/saved_model/reader.cc:52] Reading SavedModel from: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpicqrm61q
2023-05-31 18:21:19.694012: I tensorflow/cc/saved_model/reader.cc:106] Reading meta graph with tags { serve }
2023-05-31 18:21:19.694028: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpicqrm61q
2023-05-31 18:21:19.702016: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-05-31 18:21:19.785080: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

TFLite model medium : 
Prediction value of test sample with ID 7: 0.9571979
Accuracy: 0.9877431906614786
Precision: 0.9503722084367245
Recall: 0.9973958333333334


1/1 [==============================] - 0s 28ms/step
TF model medium : 
Prediction value of test sample with ID 7: 0.9571979
Accuracy: 0.9877431988716125
Precision: 0.9503722190856934
Recall: 0.9973958134651184
INFO:tensorflow:Assets written to: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpa0tfktvz/assets


INFO:tensorflow:Assets written to: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpa0tfktvz/assets
2023-05-31 18:21:25.454543: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-31 18:21:25.454566: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-31 18:21:25.454849: I tensorflow/cc/saved_model/reader.cc:52] Reading SavedModel from: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpa0tfktvz
2023-05-31 18:21:25.457837: I tensorflow/cc/saved_model/reader.cc:106] Reading meta graph with tags { serve }
2023-05-31 18:21:25.457860: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpa0tfktvz
2023-05-31 18:21:25.467728: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-05-31 18:21:25.559990: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

TFLite model large : 
Prediction value of test sample with ID 7: 0.9656337
Accuracy: 0.9873540856031129
Precision: 0.9488026424442609
Recall: 0.9973958333333334


1/1 [==============================] - 0s 29ms/step
TF model large : 
Prediction value of test sample with ID 7: 0.96563363
Accuracy: 0.9873540997505188
Precision: 0.948802649974823
Recall: 0.9973958134651184
INFO:tensorflow:Assets written to: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpxbnt_8mv/assets


INFO:tensorflow:Assets written to: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpxbnt_8mv/assets
/Users/gabrielesavoia/.pyenv/versions/3.8.5/envs/env-uni-iot/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2023-05-31 18:21:30.391762: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-31 18:21:30.391784: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-05-31 18:21:30.392073: I tensorflow/cc/saved_model/reader.cc:52] Reading SavedModel from: /var/folders/z3/yl30gjf55_x2qqgttnn1gjjc0000gn/T/tmpxbnt_8mv
2023-05-31 18:21:30.393948: I tensorflow/cc/saved_model/reader.cc:106] Reading meta graph with tags { serve }
2023-05-31 18:21:30.393965: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folde

TF model large_quant : 
Prediction value of test sample with ID 7: 0.93359375
Accuracy: 0.9511673151750972
Precision: 0.9611054247697032
Recall: 0.8151041666666666


**Il risultato che ottengo nelle schede deve essere uguale al valore stampato sopra in relazione a "Prediction value of test sample with ID .... "**